In [1]:
# pip install openai langchain neo4j speechrecognition pandas
# pip install ipykernel
# pip install langchain-openai
# pip install --upgrade langchain langchain-community langchain-openai
# pip install python-dotenv
# pip install python-docx
# pip install faiss-cpu openai
# pip install flask
# pip install python-docx
# pip install pyaudio
# pip install pyttsx3
# 
# 
# 






In [2]:
# !pip install --upgrade python-dotenv

In [3]:
# ai_policy_compliance.ipynb




import openai
import neo4j
import langchain
import speech_recognition as sr
import pandas as pd
from neo4j import GraphDatabase
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain.vectorstores import Neo4jVector

In [4]:
import os

# Change to your correct project directory
os.chdir("/Users/drashteeparmar/Drashtee/Programming/AI_Powered_Policy_Compliance_System/")

# Verify the new working directory
print("New Directory:", os.getcwd())

print(".env Exists:", os.path.exists(".env"))

New Directory: /Users/drashteeparmar/Drashtee/Programming/AI_Powered_Policy_Compliance_System
.env Exists: True


In [5]:
import os
from dotenv import load_dotenv, dotenv_values

# Explicitly specify the .env file path
dotenv_path = os.path.abspath(".env")

# Load the .env file
load_dotenv(dotenv_path=dotenv_path)

# Debugging: Read .env without loading it into os.environ
config = dotenv_values(dotenv_path)
# print("Config Output:", config)  # Should print key-value pairs

# Retrieve API Key
api_key = os.getenv("OPENAI_API_KEY")
print("OpenAI API Key:", "Loaded successfully!" if api_key else "Not found!")

OpenAI API Key: Loaded successfully!


In [6]:
import os
from dotenv import load_dotenv
from neo4j import GraphDatabase

# Load environment variables
load_dotenv()

# Retrieve Neo4j credentials
NEO4J_URI = os.getenv("NEO4J_URI")
NEO4J_USER = os.getenv("NEO4J_USER")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")

# Debugging: Print variables to confirm they are loaded
print("🔍 Checking Neo4j Credentials:")
print(f"✅ Neo4j URI: {NEO4J_URI}")
print(f"✅ Neo4j User: {NEO4J_USER}")
print(f"✅ Neo4j Password Loaded: {'Yes' if NEO4J_PASSWORD else 'No'}")

# Test connection
try:
    driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))
    with driver.session() as session:
        session.run("RETURN 1")
    print("✅ Successfully connected to Neo4j!")
except Exception as e:
    print("❌ Error connecting to Neo4j:", e)

🔍 Checking Neo4j Credentials:
✅ Neo4j URI: bolt://localhost:7687
✅ Neo4j User: compliance
✅ Neo4j Password Loaded: Yes
✅ Successfully connected to Neo4j!


Create and Insert Data in Neo4j

In [7]:
# pip install python-docx

In [8]:
import docx
print("✅ python-docx is installed and working!")

✅ python-docx is installed and working!


In [9]:
import os

policies_dir = "policies"

# Check if the directory exists
if os.path.exists(policies_dir) and os.path.isdir(policies_dir):
    print(f"✅ Directory '{policies_dir}' exists!")
    print("📂 Contents:", os.listdir(policies_dir))
else:
    print(f"❌ Directory '{policies_dir}' not found!")
    print("📂 Current Working Directory:", os.getcwd())

✅ Directory 'policies' exists!
📂 Contents: ['6. Foreign Transaction Fees and Currency Conversion Policy.docx', '1. Cardholder Agreement Terms.docx', '8. Promotional Offers and Balance Transfers Policy .docx', '5. Dispute Resolution Policy.docx', '4. Rewards Program Terms.docx', '9. Privacy and Data Sharing Policy.docx', '3. Fraud Protection and Chargeback Policy.docx', '10. Legal and Compliance Policy.docx', '2. Billing & Payment Policy.docx', '7. Card Benefits (Travel Insurance, Purchase Protection, and Warranty).docx']


In [10]:
import os
from docx import Document
from neo4j import GraphDatabase

class PolicyDatabase:
    def __init__(self, uri, user, password):
        """Initialize connection to Neo4j."""
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        self.driver.close()
        print("🔻 Neo4j connection closed.")

    def initialize_schema(self):
        print("🔄 Creating Neo4j schema...")
        with self.driver.session() as session:
            session.run("CREATE CONSTRAINT IF NOT EXISTS FOR (p:Policy) REQUIRE p.id IS UNIQUE;")
            print("✅ Schema created successfully!")

    def insert_policies_from_directory(self, directory):
        print(f"📂 Searching for policies in '{directory}'...")

        if not os.path.exists(directory):
            print(f"❌ Directory '{directory}' not found!")
            return

        policy_id = 1
        with self.driver.session() as session:
            for file in os.listdir(directory):
                if file.endswith(".docx"):
                    file_path = os.path.join(directory, file)
                    print(f"📖 Processing: {file_path}")

                    # Read document content
                    doc = Document(file_path)
                    content = "\n".join([para.text for para in doc.paragraphs])

                    # Insert into Neo4j
                    session.run("""
                        MERGE (p:Policy {id: $id})
                        SET p.name = $name, p.content = $content
                    """, id=policy_id, name=file, content=content)
                    
                    print(f"✅ Inserted: {file}")
                    policy_id += 1
    
    def search_policies(self, keyword):
        """Search for policies based on keywords."""
        print(f"🔍 Searching for policies with keyword: '{keyword}'...")
        with self.driver.session() as session:
            result = session.run("""
                MATCH (p:Policy)
                WHERE p.content CONTAINS $keyword
                RETURN p.name, p.content LIMIT 5;
            """, keyword=keyword)

            policies = result.data()
            if policies:
                for policy in policies:
                    print(f"✅ Found: {policy['p.name']}")
                    print(f"📄 Content: {policy['p.content'][:200]}...\n")
            else:
                print("❌ No policies found!")

    def ask_policy_ai(self, question):
        """Use AI (OpenAI) to analyze policy documents and answer questions."""
        print(f"🧠 AI is analyzing: '{question}'...")

        # Retrieve all policy content
        with self.driver.session() as session:
            result = session.run("MATCH (p:Policy) RETURN p.content;")
            policies = [record["p.content"] for record in result]

        if not policies:
            print("❌ No policies found!")
            return

        # Combine all policies into one prompt
        context = "\n\n".join(policies)
        prompt = f"Based on the following policies, answer: {question}\n\n{context}"

        # Send query to OpenAI API
        response = openai.ChatCompletion.create(
            model="gpt-4",
            messages=[{"role": "system", "content": "You are a policy expert."},
                      {"role": "user", "content": prompt}]
        )

        answer = response["choices"][0]["message"]["content"]
        print("🤖 AI Response:", answer)


# # Initialize and set up the database
# db = PolicyDatabase(NEO4J_URI, NEO4J_USER, NEO4J_PASSWORD)
# db.initialize_schema()
# db.insert_policies_from_directory("policies")  # Load all policy files
# db.search_policies("refund")  # Example: Search for "refund" in policies
# db.ask_policy_ai("What is the refund policy?")  # AI-based query
# db.close()

# Initialize and set up the database
db = PolicyDatabase(NEO4J_URI, NEO4J_USER, NEO4J_PASSWORD)
db.initialize_schema()
db.insert_policies_from_directory("policies")  # Load all policy files
db.close()

🔄 Creating Neo4j schema...
✅ Schema created successfully!
📂 Searching for policies in 'policies'...
📖 Processing: policies/6. Foreign Transaction Fees and Currency Conversion Policy.docx
✅ Inserted: 6. Foreign Transaction Fees and Currency Conversion Policy.docx
📖 Processing: policies/1. Cardholder Agreement Terms.docx
✅ Inserted: 1. Cardholder Agreement Terms.docx
📖 Processing: policies/8. Promotional Offers and Balance Transfers Policy .docx
✅ Inserted: 8. Promotional Offers and Balance Transfers Policy .docx
📖 Processing: policies/5. Dispute Resolution Policy.docx
✅ Inserted: 5. Dispute Resolution Policy.docx
📖 Processing: policies/4. Rewards Program Terms.docx
✅ Inserted: 4. Rewards Program Terms.docx
📖 Processing: policies/9. Privacy and Data Sharing Policy.docx
✅ Inserted: 9. Privacy and Data Sharing Policy.docx
📖 Processing: policies/3. Fraud Protection and Chargeback Policy.docx
✅ Inserted: 3. Fraud Protection and Chargeback Policy.docx
📖 Processing: policies/10. Legal and Comp

Please check those policies on:

http://localhost:7474/browser/

Query: MATCH (p:Policy) RETURN p.id, p.name, LEFT(p.content, 100) ;

In [11]:
# pip install --upgrade openai



In [12]:
from openai import OpenAI

class PolicyDatabase:
    def __init__(self, uri, user, password):
        """Initialize connection to Neo4j."""
        self.driver = GraphDatabase.driver(uri, auth=(user, password))
        self.client = OpenAI()  # Initialize OpenAI Client

    def close(self):
        """Close the Neo4j connection."""
        self.driver.close()
        print("🔻 Neo4j connection closed.")

    def ask_policy_ai(self, question):
        """Use AI (OpenAI) to analyze policy documents and answer questions."""
        print(f"🧠 AI is analyzing: '{question}'...")

        # Retrieve all policy content
        with self.driver.session() as session:
            result = session.run("MATCH (p:Policy) RETURN p.content;")
            policies = [record["p.content"] for record in result]

        if not policies:
            print("❌ No policies found!")
            return None

        # Combine all policies into one prompt
        context = "\n\n".join(policies)
        prompt = f"Based on the following policies, answer: {question}\n\n{context}"

        print("📨 Sending request to OpenAI...")
        print(f"🔹 Prompt (First 500 chars):\n{prompt[:500]}...")

        try:
            # New OpenAI API format
            response = self.client.chat.completions.create(
                model="gpt-4",
                messages=[
                    {"role": "system", "content": "You are a policy expert. Answer questions based on policies."},
                    {"role": "user", "content": prompt}
                ]
            )

            print("📩 Received response from OpenAI API:")
            print(response)  # Print full API response for debugging

            # Extract answer
            answer = response.choices[0].message.content
            print("🤖 AI Response:", answer)
            return answer

        except Exception as e:
            print("❌ Error calling OpenAI API:", e)

In [13]:
import openai
import os
from dotenv import load_dotenv

# Load API key from .env
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

class PolicyDatabase:
    # ... (other methods)

    def ask_policy_ai(self, question):
        """Use AI (OpenAI) to analyze policy documents and answer questions."""
        print(f"🧠 AI is analyzing: '{question}'...")

        # Retrieve all policy content
        with self.driver.session() as session:
            result = session.run("MATCH (p:Policy) RETURN p.content;")
            policies = [record["p.content"] for record in result]

        if not policies:
            print("❌ No policies found!")
            return

        # Combine all policies into one prompt
        context = "\n\n".join(policies)
        prompt = f"Based on the following policies, answer: {question}\n\n{context}"

        # New OpenAI API format
        response = openai.ChatCompletion.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": "You are a policy expert. Answer questions based on policies."},
                {"role": "user", "content": prompt}
            ]
        )

        answer = response["choices"][0]["message"]["content"]
        print("🤖 AI Response:", answer)

In [14]:
import openai
import os
from dotenv import load_dotenv

# Load API key from .env
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

class PolicyDatabase:
    # ... (previous methods)

    def ask_policy_ai(self, question):
        """Use AI (OpenAI) to analyze policy documents and answer questions."""
        print(f"🧠 AI is analyzing: '{question}'...")

        # Retrieve all policy content
        with self.driver.session() as session:
            result = session.run("MATCH (p:Policy) RETURN p.content;")
            policies = [record["p.content"] for record in result]

        if not policies:
            print("❌ No policies found!")
            return None

        # Combine all policies into one prompt
        context = "\n\n".join(policies)
        prompt = f"Based on the following policies, answer: {question}\n\n{context}"

        print("📨 Sending request to OpenAI...")
        print(f"🔹 Prompt:\n{prompt[:500]}...")  # Print first 500 chars of the prompt

        try:
            # Correct OpenAI API format
            response = openai.ChatCompletion.create(
                model="gpt-4",
                messages=[
                    {"role": "system", "content": "You are a policy expert. Answer questions based on policies."},
                    {"role": "user", "content": prompt}
                ]
            )

            print("📩 Received response from OpenAI API:")
            print(response)  # Print full API response for debugging

            # Extract answer
            answer = response["choices"][0]["message"]["content"]

            print("🤖 AI Response:", answer)
            return answer

        except Exception as e:
            print("❌ Error calling OpenAI API:", e)
            return None

In [16]:
import os
import openai
import faiss
import numpy as np
import speech_recognition as sr
from docx import Document
from neo4j import GraphDatabase
from dotenv import load_dotenv
from openai import OpenAI

# Load API keys from .env
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
NEO4J_URI = os.getenv("NEO4J_URI")
NEO4J_USER = os.getenv("NEO4J_USER")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")

# Initialize OpenAI client
client = OpenAI(api_key=openai_api_key)

class PolicyDatabase:
    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        self.driver.close()
        print("🔻 Neo4j connection closed.")

    def extract_text_from_docx(self, file_path):
        try:
            doc = Document(file_path)
            return "\n".join([para.text for para in doc.paragraphs])
        except Exception as e:
            print(f"❌ Error reading {file_path}: {e}")
            return ""

    def insert_policies_from_directory(self, directory):
        print(f"📂 Searching for policies in '{directory}'...")
        if not os.path.exists(directory):
            print(f"❌ Directory '{directory}' not found!")
            return
        
        policy_id = 1
        with self.driver.session() as session:
            for file in os.listdir(directory):
                if file.endswith(".docx"):
                    file_path = os.path.join(directory, file)
                    print(f"📖 Processing: {file_path}")
                    content = self.extract_text_from_docx(file_path)
                    if not content.strip():
                        print(f"⚠️ Skipping empty document: {file}")
                        continue
                    
                    session.run("""
                        MERGE (p:Policy {id: $id})
                        SET p.name = $name, p.content = $content
                    """, id=policy_id, name=file, content=content)
                    print(f"✅ Inserted: {file}")
                    policy_id += 1

    def ask_policy_ai(self, question):
        print(f"🧠 AI is analyzing: '{question}'...")
        with self.driver.session() as session:
            result = session.run("MATCH (p:Policy) RETURN p.content;")
            policies = [record["p.content"] for record in result]
        
        if not policies:
            print("❌ No policies found in the database!")
            return None
        
        context = "\n\n".join(policies[:3])  # Limit policies sent to OpenAI
        prompt = f"Based on the following policies, answer: {question}\n\n{context}"
        
        print("📨 Sending request to OpenAI...")
        try:
            response = client.chat.completions.create(
                model="gpt-4",
                messages=[
                    {"role": "system", "content": "You are a policy expert. Answer questions based on policies."},
                    {"role": "user", "content": prompt}
                ]
            )
            
            answer = response.choices[0].message.content
            print("🤖 AI Response:", answer)
            return answer
        except Exception as e:
            print("❌ Error calling OpenAI API:", e)
            return None

# Speech-to-Text Function
def get_speech_input():
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        print("🎤 Listening... Speak your policy-related question:")
        try:
            audio = recognizer.listen(source)
            query = recognizer.recognize_google(audio)
            print(f"🗣️ You asked: {query}")
            return query
        except sr.UnknownValueError:
            print("❌ Speech Recognition could not understand the audio")
            return None
        except sr.RequestError:
            print("❌ Could not request results, check your internet connection")
            return None

# Initialize database
policy_db = PolicyDatabase(NEO4J_URI, NEO4J_USER, NEO4J_PASSWORD)
policy_db.insert_policies_from_directory("policies")

# Use speech input to ask a policy question
spoken_question = get_speech_input()
if spoken_question:
    policy_db.ask_policy_ai(spoken_question)

policy_db.close()

📂 Searching for policies in 'policies'...
📖 Processing: policies/6. Foreign Transaction Fees and Currency Conversion Policy.docx
✅ Inserted: 6. Foreign Transaction Fees and Currency Conversion Policy.docx
📖 Processing: policies/1. Cardholder Agreement Terms.docx
✅ Inserted: 1. Cardholder Agreement Terms.docx
📖 Processing: policies/8. Promotional Offers and Balance Transfers Policy .docx
✅ Inserted: 8. Promotional Offers and Balance Transfers Policy .docx
📖 Processing: policies/5. Dispute Resolution Policy.docx
✅ Inserted: 5. Dispute Resolution Policy.docx
📖 Processing: policies/4. Rewards Program Terms.docx
✅ Inserted: 4. Rewards Program Terms.docx
📖 Processing: policies/9. Privacy and Data Sharing Policy.docx
✅ Inserted: 9. Privacy and Data Sharing Policy.docx
📖 Processing: policies/3. Fraud Protection and Chargeback Policy.docx
✅ Inserted: 3. Fraud Protection and Chargeback Policy.docx
📖 Processing: policies/10. Legal and Compliance Policy.docx
✅ Inserted: 10. Legal and Compliance Po

In [ ]:
# pip install pyttsx3

Note: you may need to restart the kernel to use updated packages.


In [18]:
# import speech_recognition as sr

# for index, name in enumerate(sr.Microphone.list_microphone_names()):
#     print(f"🎤 Microphone {index}: {name}")

In [19]:
# import os
# import openai
# import faiss
# import numpy as np
# import speech_recognition as sr
# import pyttsx3
# from docx import Document
# from neo4j import GraphDatabase
# from dotenv import load_dotenv
# from openai import OpenAI

# # Load API keys from .env
# load_dotenv()
# openai_api_key = os.getenv("OPENAI_API_KEY")
# NEO4J_URI = os.getenv("NEO4J_URI")
# NEO4J_USER = os.getenv("NEO4J_USER")
# NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")

# # Initialize OpenAI client
# client = OpenAI(api_key=openai_api_key)

# # Initialize Text-to-Speech Engine
# tts_engine = pyttsx3.init()

# def speak_text(text):
#     """Convert text to speech."""
#     tts_engine.say(text)
#     tts_engine.runAndWait()

# def log_conversation(question, answer):
#     """Save the conversation in a text file."""
#     with open("conversation_log.txt", "a") as file:
#         file.write(f"Q: {question}\nA: {answer}\n---\n")

# class PolicyDatabase:
#     def __init__(self, uri, user, password):
#         self.driver = GraphDatabase.driver(uri, auth=(user, password))

#     def close(self):
#         self.driver.close()
#         print("🔻 Neo4j connection closed.")

#     def extract_text_from_docx(self, file_path):
#         try:
#             doc = Document(file_path)
#             return "\n".join([para.text for para in doc.paragraphs])
#         except Exception as e:
#             print(f"❌ Error reading {file_path}: {e}")
#             return ""

#     def insert_policies_from_directory(self, directory):
#         print(f"📂 Searching for policies in '{directory}'...")
#         if not os.path.exists(directory):
#             print(f"❌ Directory '{directory}' not found!")
#             return
        
#         policy_id = 1
#         with self.driver.session() as session:
#             for file in os.listdir(directory):
#                 if file.endswith(".docx"):
#                     file_path = os.path.join(directory, file)
#                     print(f"📖 Processing: {file_path}")
#                     content = self.extract_text_from_docx(file_path)
#                     if not content.strip():
#                         print(f"⚠️ Skipping empty document: {file}")
#                         continue
                    
#                     session.run("""
#                         MERGE (p:Policy {id: $id})
#                         SET p.name = $name, p.content = $content
#                     """, id=policy_id, name=file, content=content)
#                     print(f"✅ Inserted: {file}")
#                     policy_id += 1

#     def ask_policy_ai(self, question):
#         print(f"🧠 AI is analyzing: '{question}'...")
#         with self.driver.session() as session:
#             result = session.run("MATCH (p:Policy) RETURN p.content;")
#             policies = [record["p.content"] for record in result]
        
#         if not policies:
#             print("❌ No policies found in the database!")
#             return None
        
#         context = "\n\n".join(policies[:3])  # Limit policies sent to OpenAI
#         prompt = f"Based on the following policies, answer: {question}\n\n{context}"
        
#         print("📨 Sending request to OpenAI...")
#         try:
#             response = client.chat.completions.create(
#                 model="gpt-4",
#                 messages=[
#                     {"role": "system", "content": "You are a policy expert. Answer questions based on policies."},
#                     {"role": "user", "content": prompt}
#                 ]
#             )
            
#             answer = response.choices[0].message.content
#             print("🤖 AI Response:", answer)
#             speak_text(answer)  # AI speaks the response
#             log_conversation(question, answer)  # Log Q&A
#             self.store_conversation_in_db(question, answer)  # Store in Neo4j
#             return answer
#         except Exception as e:
#             print("❌ Error calling OpenAI API:", e)
#             return None
    
#     def store_conversation_in_db(self, question, answer):
#         """Save the conversation in Neo4j."""
#         with self.driver.session() as session:
#             session.run("""
#                 CREATE (c:Conversation {question: $question, answer: $answer})
#             """, question=question, answer=answer)
#         print("✅ Conversation stored in Neo4j")

# # Speech-to-Text Function
# def get_speech_input():
#     recognizer = sr.Recognizer()
#     with sr.Microphone() as source:
#         print("🎤 Listening... Speak your policy-related question:")
#         try:
#             audio = recognizer.listen(source)
#             query = recognizer.recognize_google(audio)
#             print(f"🗣️ You asked: {query}")
#             return query
#         except sr.UnknownValueError:
#             print("❌ Speech Recognition could not understand the audio")
#             return None
#         except sr.RequestError:
#             print("❌ Could not request results, check your internet connection")
#             return None

# # Initialize database
# policy_db = PolicyDatabase(NEO4J_URI, NEO4J_USER, NEO4J_PASSWORD)
# policy_db.insert_policies_from_directory("policies")

# # Continuous conversation loop
# while True:
#     spoken_question = get_speech_input()
#     if spoken_question:
#         if "exit" in spoken_question.lower():
#             print("👋 Exiting conversation.")
#             speak_text("Goodbye! Have a great day!")
#             break
#         policy_db.ask_policy_ai(spoken_question)

# policy_db.close()


In [20]:
import os
import openai
import faiss
import numpy as np
import speech_recognition as sr
import pyttsx3
from docx import Document
from neo4j import GraphDatabase, exceptions
from dotenv import load_dotenv
from openai import OpenAI

# Load API keys from .env
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
NEO4J_URI = os.getenv("NEO4J_URI")
NEO4J_USER = os.getenv("NEO4J_USER")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")

# Initialize OpenAI client
client = OpenAI(api_key=openai_api_key)

# Initialize Text-to-Speech Engine
tts_engine = pyttsx3.init()

def speak_text(text):
    """Convert text to speech."""
    tts_engine.say(text)
    tts_engine.runAndWait()

def log_conversation(question, answer):
    """Save the conversation in a text file."""
    with open("conversation_log.txt", "a") as file:
        file.write(f"Q: {question}\nA: {answer}\n---\n")

class PolicyDatabase:
    def __init__(self, uri, user, password):
        try:
            self.driver = GraphDatabase.driver(uri, auth=(user, password))
            # Test connection
            with self.driver.session() as session:
                session.run("RETURN 1")
            print("✅ Connected to Neo4j successfully!")
        except exceptions.ServiceUnavailable as e:
            print("❌ Error: Unable to connect to Neo4j. Check if Neo4j is running and credentials are correct.")
            self.driver = None
        except exceptions.AuthError as e:
            print("❌ Authentication Error: Invalid username or password for Neo4j.")
            self.driver = None

    def close(self):
        if self.driver:
            self.driver.close()
            print("🔻 Neo4j connection closed.")

    def ask_policy_ai(self, question):
        if not self.driver:
            print("❌ Cannot query AI: No database connection.")
            return None

        print(f"🧠 AI is analyzing: '{question}'...")
        with self.driver.session() as session:
            result = session.run("MATCH (p:Policy) RETURN p.content;")
            policies = [record["p.content"] for record in result]
        
        if not policies:
            print("❌ No policies found in the database!")
            return None
        
        context = "\n\n".join(policies[:3])  # Limit policies sent to OpenAI
        prompt = f"Based on the following policies, answer: {question}\n\n{context}"
        
        print("📨 Sending request to OpenAI...")
        try:
            response = client.chat.completions.create(
                model="gpt-4",
                messages=[
                    {"role": "system", "content": "You are a policy expert. Answer questions based on policies."},
                    {"role": "user", "content": prompt}
                ]
            )
            
            answer = response.choices[0].message.content
            print("🤖 AI Response:", answer)
            speak_text(answer)  # AI speaks the response
            log_conversation(question, answer)  # Log Q&A
            self.store_conversation_in_db(question, answer)  # Store in Neo4j
            return answer
        except Exception as e:
            print("❌ Error calling OpenAI API:", e)
            return None

# Initialize database
policy_db = PolicyDatabase(NEO4J_URI, NEO4J_USER, NEO4J_PASSWORD)

if policy_db.driver:
    # Continuous conversation loop
    while True:
        spoken_question = get_speech_input()
        if spoken_question:
            if "exit" in spoken_question.lower():
                print("👋 Exiting conversation.")
                speak_text("Goodbye! Have a great day!")
                break
            policy_db.ask_policy_ai(spoken_question)
    
    policy_db.close()
else:
    print("❌ Database connection failed. Exiting...")


✅ Connected to Neo4j successfully!
🎤 Listening... Speak your policy-related question:
🗣️ You asked: how much interest charge for late
🧠 AI is analyzing: 'how much interest charge for late'...
📨 Sending request to OpenAI...
🤖 AI Response: Based on Policy 1: Cardholder Agreement Terms, if a payment is late, the cardholder is required to pay a late fee of $39. Additionally, interest begins to accrue on the outstanding balance. The standard Annual Percentage Rate (APR) for purchases is 19.99%, which applies to any unpaid balance after the billing cycle ends. If a cardholder misses two consecutive payments, the APR will increase to 29.99%. The penalty APR is applied until six consecutive on-time payments are made, at which point the rate will revert to the standard APR.
On the other hand, Policy 6: Foreign Transaction Fees and Currency Conversion Policy states that cash withdrawals from foreign ATMs begin accruing interest immediately at an APR of 24.99%. There is no grace period for cash a